In [28]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

## Import specific libraries needed
import pydicom
import glob

# Mathematical
import scipy

from keras.models import model_from_json
from skimage.transform import resize

# Library to create imgae path
import pathlib
import imageio
## End of library to images


from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
from keras.preprocessing import image

# Neccessary for confusion matrix
import sklearn.metrics

Incorrect path code

My own attempt to extract the files in my terms

In [29]:
# Glob the test data and load a single image path
#test_paths = pathlib.Path( '../input/stage1_train').glob('*/images/*.dcm')
#test_sorted = sorted([x for x in test_paths])
#img_path = test_sorted[1]
# img = imageio.imread(str(im_path))
#img = pydicom.dcmread(img_path)


The problem with the code above is that `test_paths` is empty because the generator is not well created. Indeed in my workspace there is no /image/ folder and also my pattern in the Path object is not correct ('.' instead of '/')

In [30]:
test_paths = pathlib.Path('.').glob('*.dcm')
test_sorted = sorted([x for x in test_paths])

In [31]:
test_sorted


[PosixPath('test1.dcm'),
 PosixPath('test2.dcm'),
 PosixPath('test3.dcm'),
 PosixPath('test4.dcm'),
 PosixPath('test5.dcm'),
 PosixPath('test6.dcm')]

In [32]:
# Pull one image with the index
im_path = test_sorted[1]
# This code read that image into an array
im = imageio.imread(str(im_path))

In [33]:
im

Array([[202, 199, 195, ...,   5,   2,   0],
       [199, 196, 195, ...,   5,   2,   0],
       [196, 194, 193, ...,   5,   2,   0],
       ...,
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 254, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0]], dtype=uint8)

### Needs all the dicom files here in a list
Reference to the excercise on Lesson 3:14


In [34]:
## Inserted Col
## First, read all of my DICOM files into a list
test_dicoms = glob.glob("*.dcm")

In [35]:
test_dicoms

['test2.dcm', 'test5.dcm', 'test1.dcm', 'test4.dcm', 'test6.dcm', 'test3.dcm']

**Let's look at the contents of the first DICOM;**

In [36]:
# Inserted
dcm1 = pydicom.dcmread(test_dicoms[0])

In [37]:
dcm1

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.153751009835107614666834563294684339746480
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'Cardiomegaly'
(0010, 0020) Patient ID                          LO: '1'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '58'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'AP'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.111396399361969898205364400549799252857604
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.195628213694300498946760767481291263511724
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows     

Looking at the attributes listed above, I can see that I'm looking to extract the following attributes:

* Modality
* Study Description
* Patient ID
* Patient's Sex
* Patient's Age
* Rows
* Columns

In [38]:
# Lets extract and read the first file
dcm1 = pydicom.dcmread(test_dicoms[0])

In [39]:
# Lets Aacess value from a field
dcm1.Modality

'DX'

In [40]:
# Access another value
dcm1.StudyDescription

'Cardiomegaly'

In [41]:
dcm1.PatientID

'1'

In [42]:
dcm1.PatientSex

'M'

In [43]:
dcm1.PatientAge

'58'

In [44]:
dcm1.BodyPartExamined

'CHEST'

### The purpose of check_dicom
The check_dicom function reads in a .dcm file, checks the important fields for our device, and returns a numpy array of just the imaging data.

The function **[check_dicom()]**, is required to check the image type(MODALITY), Body Part Examined (Chest), and Image position (PA or AP ) for each DICOM image and check if the input to the algorithm is valid to be predicted by our algorithm or not.

However, your function make use of all dicoms files at once which isn't what we want to achieve here. The function validates a dicom file at a time, so you need to read the dicom using the filename provided to you as a parameter. From there on, you proceed on evaluating the fields making sure they contain the right values before sending the image for further processing and prediction.

Ensure body part examined (BodyPartExamined) is CHEST else invalid
Also ensure the Image Position (PatientPosition) is either AP or PA anything else is invalid.
If everything is okay, you can return the image with this code segment which is already given to you by default.

Consider this as a hint Applying the above check-in your program it should correctly predict the first three images and should reject the last three images

In [88]:
# First
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
  
        
    # use this function to read a single dicom file as you have already # done in your code
    dcm = pydicom.dcmread(filename) 
    img = dcm.pixel_array
    
    print('\nLoad file {} ...'.format(filename))
    print('Patient ID: ', dcm.PatientID)
    print('Patient Position: ', dcm.PatientPosition)
    print('Patient Age: ', dcm.PatientAge)
    print('Patient Sex: ', dcm.PatientSex)
    print('Body Part Examined: ', dcm.BodyPartExamined)
    print('Study Description: ', dcm.StudyDescription)
    print('Image type: ', dcm.Modality)
    
        
    
    # check fields for required values e.g
    if dcm.Modality != "DX":
        print('This image is excluded due to invalid Modality for prediction: ', dcm.Modality)
        return None

    # Ensure body part examined (BodyPartExamined) is CHEST else invalid
    if dcm.BodyPartExamined != "CHEST":
        print('This image is excluded due to invalid Body Part Examined for prediction: ', dcm.BodyPartExamined)
        return None
        
    # Also ensure the Image Position (PatientPosition) is either AP or PA anything else is invalid.
    if (dcm.PatientPosition != "AP") & (dcm.PatientPosition != "PA"):
        print('This image is excluded due to invalid Patient Position for prediction: ', dcm.PatientPosition)
        return None
        

        
            
    return img


# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,IMG_SIZE): # Remove img_mean, img_std and img_size from parameters
    # todo
    IMG_SIZE = (1,224, 224, 3)

    # Standardize it using the standardization formula:(X - X_mean)/X_std_dev
    proc_img = (img - img_mean)/img_std
    # Finally resizing it to our desired size as a marker.
    # proc_img = proc_img.resize(img, IMG_SIZE)
    proc_img = np.resize(proc_img, IMG_SIZE)
    
    
    return proc_img

    
    
# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    # todo
    
    model_path = "my_model.json" #path to saved model
    weight_path = "pneumonia_class_my_model.best.hdf5" #path to saved best weights
    
    
    # load json file 
    json_file = open(model_path, 'r')
    # read the json file
    json_to_model = json_file.read()
    json_file.close()
    my_model = model_from_json(json_to_model)
    # load weights
    my_model.load_weights(weight_path)
    
    
    # The model your return should be an actual model
    return my_model



# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(my_model, img, YOUR_THRESHOLD, numericLabels=False): 
    # todo  

    result = my_model.predict(img)
    # Use the index to retreive the value of the prediction  as result has index of o and a value.
    predict =result[0]
    if numericLabels:
        if (predict > YOUR_THRESHOLD):
            prediction = 1
        else:
            prediction = 0
    else:
        if(predict > YOUR_THRESHOLD):
            prediction = 'Prediction: Positive'
            
        else:
            prediction = 'Prediction: Negative'
    
    return prediction






### load_model function
The function load_model above doesn't load any model. 
It defines some variables and returns an empty variable. 
Trying to call the function results in the error below. 
The model you return should be an actual model.So I have changed the word model to my_model.


### To correct model_1_input error

**ValueError: Error when checking input: expected model_1_input to have 4 dimensions, but got array with shape (224, 224)** 

The model expect input tensors of 4 dimension (N, H, W, C) which stands for batch size, height, width, and channel. Therefore, to run prediction on single image, you can use numpy.expand_dims() to add dimensions to make it into (1,224,224,1). 

Another way is to use:

np.reshape(image, (1,224,224,1)). 

The solution to the above was to change IMG_SIZE to (1,224, 224, 3) and adding these tow lines of code

```
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
```



In [89]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

## These files are located at the root folder /my_model.json and /pneumonia_class_mmy_model.best.hdf5
model_path = 'my_model.json' #path to saved model
weight_path = '"pneumonia_class_my_model.best.hdf5"' #path to saved best weights


# IMG_SIZE=224 # This might be different if you did not use vgg16
# img_mean = 0 # loads the mean image value they used during training preprocessing
# img_std = 1 # loads the std dev image value they used during training preprocessing


my_model = load_model(model_path, weight_path) #loads model 
best_threshold = 0.40 # Used for model classification
YOUR_THRESHOLD = best_threshold

# Use the dicocm files to test your prediction
#for i in test_dicoms:

for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i) 
  
    if img is None:
        continue
 
    print('Image shape: {}'.format(img.shape))
    img_reshape = img.reshape((img.shape[0], img.shape[1]))
    
    # Declare the mean and std here
    img_mean = np.mean(img)
    img_std = np.std(img)
    # To run prediction on single image, use numpy.expand_dims()
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    IMG_SIZE = (1,224, 224,3) # this is the shape for VGG16 model
    
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,best_threshold)
    print(pred)



Load file test1.dcm ...
Patient ID:  2
Patient Position:  PA
Patient Age:  81
Patient Sex:  M
Body Part Examined:  CHEST
Study Description:  No Finding
Image type:  DX
Image shape: (1024, 1024)
Prediction: Positive

Load file test2.dcm ...
Patient ID:  1
Patient Position:  AP
Patient Age:  58
Patient Sex:  M
Body Part Examined:  CHEST
Study Description:  Cardiomegaly
Image type:  DX
Image shape: (1024, 1024)
Prediction: Positive

Load file test3.dcm ...
Patient ID:  61
Patient Position:  AP
Patient Age:  77
Patient Sex:  M
Body Part Examined:  CHEST
Study Description:  Effusion
Image type:  DX
Image shape: (1024, 1024)
Prediction: Negative

Load file test4.dcm ...
Patient ID:  2
Patient Position:  PA
Patient Age:  81
Patient Sex:  M
Body Part Examined:  RIBCAGE
Study Description:  No Finding
Image type:  DX
This image is excluded due to invalid Body Part Examined for prediction:  RIBCAGE

Load file test5.dcm ...
Patient ID:  2
Patient Position:  PA
Patient Age:  81
Patient Sex:  M
Bod

### Clinical workflow integration
This file deals with clinical workflow integration

### Measure performance of the test files

**Now, let's create the dataframe that we want, and populate it in a loop with all of our images**

In [90]:
all_data = []

for img in test_dicoms: 
    # Define a variable and assign, a read of the file
    fields = ['img.PatientID', 'img.StudyDescription', 'img.iterrows', 'img.columns']
    all_data.append(fields)
    


In [91]:
test_df = pd.DataFrame(all_data, 
                      columns = ['PatientID', 'StudyDescription', 'Rows','Columns'])

pd.DataFrame(test_df).to_csv('test_df.csv')

**Performance in the presence of concurrent diseases**